In [89]:
import dash
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import pandas as pd
import json
from datetime import datetime, timedelta

In [90]:
df = pd.read_csv('https://raw.githubusercontent.com/ccodwg/Covid19Canada/master/timeseries_prov/active_timeseries_prov.csv')
df['date_active'].astype('str')
df=df.replace({'province': {'BC': 'British Columbia', 'NL': 'Newfoundland and Labrador', 'NWT':'Northwest Territories', 'PEI':'Prince Edward Island'}})
df['code']=df['province']
df = df.replace({'code': {'Newfoundland and Labrador':10, 'Prince Edward Island':11, 'Nova Scotia':12, 
                          'New Brunswick':13, 'Quebec':24, 'Ontario':35, 'Manitoba':53, 'Saskatchewan':47, 
                          'Alberta':48, 'British Columbia':59, 'Yukon':60, 'Northwest Territories':61, 'Nunavut':62}})

with open("georef-canada-province.geojson", "r") as geo:
    provinces = json.load(geo)

In [118]:
dates = df['date_active'].unique()
province_names = df['province'].unique()
yesterday = datetime.now() - timedelta(1)                                                                                                                                                                                   
yes = datetime.strftime(yesterday, '%d-%m-%Y')

def get_map(date):   
    df_map = df.loc[df['date_active']==date]
    map_ca = px.choropleth_mapbox(df_map, 
                           geojson=provinces, 
                           featureidkey= 'properties.prov_code',
                           locations='code', 
                           color='active_cases',
                           color_continuous_scale="Viridis",
                           range_color=(0, df_map['active_cases'].max()),
                           mapbox_style="carto-positron",
                           zoom=1, center = {"lat": 70.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'active_cases':'Active Cases'},
                          )
    map_ca.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return map_ca

def provinceData(name):
    return df.loc[df['province'] == name]


In [143]:
navbar = [       
        dbc.NavbarSimple([
            dbc.Nav([
                dbc.NavItem(dbc.NavLink("Home", active=True, href="#")),
                dbc.NavItem(dbc.NavLink("Covid 19", active=True, href="#")),
                dbc.NavItem(dbc.NavLink("Another link", active=True, href="#")),
                dbc.DropdownMenu(
                    [dbc.DropdownMenuItem("Item 1"), dbc.DropdownMenuItem("Item 2")],
                    label="Dropdown",
                    nav=True,
                ),
            ]),
            dbc.NavItem(dbc.NavLink("Page 1", href="#")),
            dbc.DropdownMenu([
                dbc.DropdownMenuItem("More pages", header=True),
                dbc.DropdownMenuItem("Page 2", href="#"),
                dbc.DropdownMenuItem("Page 3", href="#"),
            ],
            nav=True,
            # in_navbar=True,
            label="More",
            ),
        ],
        brand="NavbarSimple",
        brand_href="#",
        color="primary",
        dark=True,        
        ),
    ]

In [147]:

app = JupyterDash(__name__, external_stylesheets = [dbc.themes.BOOTSTRAP])
app.layout = html.Div([
    dbc.Row(navbar),
    dbc.Row([
        html.H1('Covid 19 in Canada'),
    ], style={'height': 150, 'background-image': 'url(https://www.who.int/images/default-source/mca/mca-covid-19/coronavirus-2.tmb-1366v.jpg)'}),
    dbc.Row([
        dbc.Col([
            # html.H6("Change the value in the text box to see callbacks in action!"),
            dbc.Row([
                html.P('Please select a date'),
                dcc.Dropdown(                
                    id='date_input',
                    options=[
                        {'label': date, 'value': date} for date in dates
                    ],
                    value=yes
                ),
            ]),
        ], width=6),
        
        dbc.Col([
            dbc.Row([
                html.P('Please select a province'),
                dcc.Dropdown(                
                    id='province_input',
                    options=[
                        {'label': province, 'value': province} for province in province_names
                    ],   
                    value='Ontario'
                ),                
            ]),
            
            dbc.Row([
                html.P('Please select Time Series'),
                dcc.Dropdown(                
                    id='timeseries_input',
                    options=[
                        {'label': 'Active Cases', 'value': 'active_cases'},
                        {'label': 'Cumulative Cases', 'value': 'cumulative_cases'},
                        {'label': 'Cumulative Recovery', 'value': 'cumulative_recovered'},
                        {'label': 'Cumulative Deaths', 'value': 'cumulative_deaths'},
                    ],
                    value='active_cases'
                ),
            ]),        
            
        ], width=6, style={}),
    ]),      
    dbc.Row([
        dbc.Col([dbc.Row(id='my-output1'),], width=6),
        dbc.Col([dbc.Row(id='my-output2'),], width=6),
    ])
])


@app.callback(
    [Output(component_id='my-output1', component_property='children'),
     Output(component_id='my-output2', component_property='children'),
    ],
    [Input(component_id='date_input', component_property='value'),
     Input(component_id='province_input', component_property='value'),
     Input(component_id='timeseries_input', component_property='value')
    ],
)

def update_output_div(selected_date, selected_province, selected_timeseries):
    map_AC = get_map(selected_date)
    if selected_timeseries == 'active_cases':
        
        line_AC = px.line(provinceData(selected_province), x='date_active', y=selected_timeseries)
        line_AC.update_xaxes(nticks=5)
        line_AC.update_yaxes(nticks=5)    
        return [
            # date_input, province_input
            dcc.Graph(figure=map_AC),
            dcc.Graph(figure=line_AC),
        ]
    else:
        bar_AC = px.bar(provinceData(selected_province), x='date_active', y=selected_timeseries)
        bar_AC.update_xaxes(nticks=5)
        bar_AC.update_yaxes(nticks=5)    
        return [
            # date_input, province_input
            dcc.Graph(figure=map_AC),
            dcc.Graph(figure=bar_AC),
        ]
        



app.run_server(mode='jupyterlab')